<a href="https://colab.research.google.com/github/Itziar23/prueba/blob/master/Copia_de_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Implementation of LadderNet using the tf.keras Functional API

from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras

drop = 0.25

def ResBlock(input_tensor, filters):
    """"
    Define Residual Block showed in Zhuang (2019)
    CONV => BATCH => RELU => DROPOUT => CONV => BATCH
      |                                   |
      |----------- SHARED LAYER ----------|
      
    """
        
    conv_1 = Conv2D(filters = filters, kernel_size = 3, padding = 'same', kernel_initializer = 'he_normal')    
    conv_1a = conv_1(input_tensor) # Shared weights conv layer
    batch_1 = BatchNormalization()(conv_1a)
    relu_1  = Activation("relu")(batch_1)
    drop_1  = Dropout(drop)(relu_1)
    conv_1b = conv_1(drop_1) # Shared weights conv layer
    batch_1 = BatchNormalization()(conv_1b)
    return batch_1

def LadderNet(input_size = (256, 256, 1), num_classes=2, filters=30):
    
    """
    LadderNet (Zhuang, 2019) implementation in tensorflow.keras
    Method: Keras Functional API
    """  
    
    # X's denote standard flow
    # XNUM denote ResBlock outputs
    
    # "First" UNet
    
    # Input branch
    inputs = Input(input_size)
    X = Conv2D(filters=filters, kernel_size=3, activation="relu", padding = 'same', kernel_initializer = 'he_normal')(inputs)

    # Down branch
    X1 = ResBlock(input_tensor=X, filters=filters) # ResBlock located in the first layer of the paper scheme
    X = Conv2D(filters=filters*2, kernel_size=3, strides=2, kernel_initializer='he_normal')(X1) 
    X = Activation("relu")(X) # This ReLU is not shown in the paper scheme
    
    X2 = ResBlock(input_tensor=X, filters=filters*2)
    X = Conv2D(filters=filters*4, kernel_size=3, strides=2, kernel_initializer='he_normal')(X2)
    X = Activation("relu")(X)
    
    X3 = ResBlock(input_tensor=X, filters=filters*4)
    X = Conv2D(filters=filters*8, kernel_size=3, strides=2, kernel_initializer='he_normal')(X3)
    X = Activation("relu")(X)
    
    X4 = ResBlock(input_tensor=X, filters=filters*8)
    X = Conv2D(filters=filters*16, kernel_size=3, strides=2, kernel_initializer='he_normal')(X4)
    X = Activation("relu")(X)
    
    # Bottom block 
    X = ResBlock(input_tensor=X, filters=filters*16)
    
    # Up branch
    X = Conv2DTranspose(filters=filters*8, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)
    X = Add()([X, X4])
    # X = Activation("relu")(X) # This ReLU is commented in the paper code
    X5 = ResBlock(input_tensor=X, filters=filters*8)
    
    X = Conv2DTranspose(filters=filters*4, kernel_size=3, strides=2, kernel_initializer='he_normal')(X5)
    X = Add()([X, X3])
    # X = Activation("relu")(X)
    X6 = ResBlock(input_tensor=X, filters=filters*4)
    
    X = Conv2DTranspose(filters=filters*2, kernel_size=3, strides=2, kernel_initializer='he_normal')(X6)
    X = Add()([X, X2])
    # X = Activation("relu")(X)
    X7 = ResBlock(input_tensor=X, filters=filters*2)
        
    X = Conv2DTranspose(filters=filters, kernel_size=3, strides=2, output_padding=1, kernel_initializer='he_normal')(X7)
    X = Add()([X, X1])
    # X = Activation("relu")(X)
    X = ResBlock(input_tensor=X, filters=filters)
    
    # Top block (bottle-neck)
    X8 = ResBlock(input_tensor=X, filters=filters)
    X = ResBlock(input_tensor=X, filters=filters)
    X = Add()([X, X8])
    
    # "Second" UNet
    
    # Down branch
    X9 = ResBlock(input_tensor=X, filters=filters)
    X = Conv2D(filters=filters*2, kernel_size=3, strides=2, kernel_initializer='he_normal')(X) 
    X = Activation("relu")(X)
    X = Add()([X7, X])    
    
    X10 = ResBlock(input_tensor=X, filters=filters*2)
    X = Conv2D(filters=filters*4, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)    
    X = Activation("relu")(X)    
    X = Add()([X6, X])
    
    X11 = ResBlock(input_tensor=X, filters=filters*4)
    X = Conv2D(filters=filters*8, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)    
    X = Activation("relu")(X)
    X = Add()([X5, X])

    X12 = ResBlock(input_tensor=X, filters=filters*8)
    X = Conv2D(filters=filters*16, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)    
    X = Activation("relu")(X)
    
    # Bottom block
    X = ResBlock(input_tensor=X, filters=filters*16)
    
    # Up branch
    X = Conv2DTranspose(filters=filters*8, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)
    X = Add()([X, X12])   
    # X = Activation("relu")(X)
    X = ResBlock(input_tensor=X, filters=filters*8)
    
    X = Conv2DTranspose(filters=filters*4, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)
    X = Add()([X, X11])
    # X = Activation("relu")(X)
    X = ResBlock(input_tensor=X, filters=filters*4)
    
    X = Conv2DTranspose(filters=filters*2, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)
    X = Add()([X, X10])
    # X = Activation("relu")(X)
    X = ResBlock(input_tensor=X, filters=filters*2)
    
    X = Conv2DTranspose(filters=filters, kernel_size=3, strides=2, kernel_initializer='he_normal', output_padding=1)(X)
    X = Add()([X, X9])
    # X = Activation("relu")(X)
    X = ResBlock(input_tensor=X, filters=filters)
    
    # Final block
    X = Conv2D(filters=num_classes, kernel_size=1, kernel_initializer='he_normal')(X)
    # X = Activation("relu")(X)
    X = Activation("softmax")(X)
    #X = Conv2D(1, 1)(X)
    
    model = Model(inputs, X)
    
    
    return model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

def conv2D_module(inputs, filters, kernel_size=3, padding="valid", pool_size=2):
    
    """
    CONV => RELU => CONV => RELU => MAXPOOL
    """
    
    x = Conv2D(filters=filters, kernel_size=kernel_size, padding=padding,
               kernel_initializer='he_normal')(inputs)
    x = Activation("relu")(x)
    x = Conv2D(filters=filters, kernel_size=kernel_size, padding=padding,
               kernel_initializer='he_normal')(inputs)
    x = Activation("relu")(x)


def UNet(input_size, depth, num_classes, filters, batch_norm):
    
    """
    UNet (Ronneberger, 2015) implementation in tensorflow.keras
    using Keras Functional API.
    """
    
    # Input layer
    inputs = Input(input_size)
    x = inputs
    
    # Encoding
    down_list = []
    for layer in range(depth):
        x = Conv2D(filters, 3, activation='relu', padding='same', kernel_initializer='he_normal')(x)
        
        if batch_norm: 
            x = BatchNormalization()(x)
            x = Conv2D(filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(x)
            x_down = BatchNormalization()(x)
        else:
            x_down = Conv2D(filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(x)
        
        down_list.append(x_down)
        x = MaxPooling2D(pool_size=2)(x_down)
        filters = filters*2
    
    # Bottom
    x = Conv2D(filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(x)
    if batch_norm: x = BatchNormalization()(x)
    x = Conv2D(filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(x)
    if batch_norm: x = BatchNormalization()(x)
    
    # Decoding
    for layer in reversed(down_list):
        filters = filters // 2
        x = UpSampling2D((2,2))(x)
        x = concatenate([x, layer])
        x = Conv2D(filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(x)
        if batch_norm: x = BatchNormalization()(x)
        x = Conv2D(filters, 3, padding = 'same', kernel_initializer = 'he_normal')(x)
        if batch_norm: x = BatchNormalization()(x)
    
    # Output layer
    x = Conv2D(filters=num_classes, kernel_size=1)(x)
    if batch_norm: x = BatchNormalization()(x)
    outputs = Activation("softmax")(x)
    
    model = Model(inputs, outputs)
    return model
    
    """
    conv1_1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1_2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)
    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    outputs = Conv2D(1, 1, activation = 'sigmoid')(conv9)
    model = Model(inputs, outputs)
    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()
    if(pretrained_weights):
    	model.load_weights(pretrained_weights)
    return model
    """